In [24]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle


In [5]:
def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

In [ ]:
# source : https://forum.step.esa.int/t/extracting-pixel-values-sentinel-2a-l2a-with-snap-or-snappy/29878/3

In [33]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

df_ksa['index'] = [x.zfill(2) for x in df_ksa['index'].astype("str")]
df_ksa['idpoint'] = df_ksa.idsegmen.astype('str') + df_ksa.idsubsegmen.astype('str') + '#' + df_ksa['index']
df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab,idpoint
0,1,1,2.381336,96.481115,01,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#01
1,1,2,2.381156,96.481115,02,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#02
2,1,3,2.380977,96.481115,03,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#03
3,1,4,2.380797,96.481115,04,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#04
4,1,5,2.380618,96.481115,05,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#05


In [34]:
# Filter only prov Jabar
df_ksa = df_ksa.query('idprov == "32"')

In [53]:
mgrs_all_ = df_ksa.MGRS.unique()
mgrs_all_

array(['48MXT', '48MYT', '48MXU', '48MXS', '48MYS', '48MZT', '48MZS',
       '49MAN', '49MAM', '49MBM', '49MBN', '49MAP', '49MBP', '48MZU',
       '48MYU'], dtype=object)

In [55]:
mgrs_ = [x for x in mgrs_all_ if x not in ['48MXU','48MYU','48MZU','48MXT']]
mgrs_

['48MYT',
 '48MXS',
 '48MYS',
 '48MZT',
 '48MZS',
 '49MAN',
 '49MAM',
 '49MBM',
 '49MBN',
 '49MAP',
 '49MBP']

In [ ]:
%time

for mgrs in mgrs_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        print(mgrs + " available")
        path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
        all_sentinel_data = glob(path_to_mgrs+"*.dim")
        df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == mgrs]
        for sent in tqdm(all_sentinel_data):
            path_to_sentinel_data = sent
            periode = path_to_sentinel_data[-21:-4]
            product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
            df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
            df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
            df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
            df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
            # print(bands_names)
            for i,r in df_ksa_mgrs_tmp.iterrows():
                val = get_values(gc, bands_names, r['lat'], r['long'])
                df_ksa_mgrs_tmp.loc[i,bands_names] = val
                # break
            # break
            df_result = pd.concat([df_result,df_ksa_mgrs_tmp])

        df_result = df_result[['idpoint','MGRS','Sigma0_VH_db','Sigma0_VV_db','periode']]
        with open('/data/ksa/03_Sampling/data/32/sampling_'+mgrs+'.pkl', 'wb') as f:
            pickle.dump(df_result, f)
    else:
        print(mgrs + " not available")
        continue
    # break

CPU times: user 10 μs, sys: 0 ns, total: 10 μs
Wall time: 20.3 μs
48MYT available


 40%|████      | 24/60 [1:54:44<3:12:03, 320.10s/it]

In [ ]:
df_result

In [52]:
with open('/data/ksa/03_Sampling/data/32/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)
df_.head()

,idpoint,MGRS,Sigma0_VH_db,Sigma0_VV_db,periode
0,320101006A1#01,48MXT,-18.347488,-12.085695,20220606_20220617
1,320101006A1#02,48MXT,-20.715675,-11.056552,20220606_20220617
2,320101006A1#03,48MXT,-19.152805,-9.379771,20220606_20220617
3,320101006A1#04,48MXT,-19.222429,-11.019622,20220606_20220617
4,320101006A1#05,48MXT,-19.996529,-9.069054,20220606_20220617


* idsegmen+idsubsegmen+#+index
* MGRS
* Sigma0_VH_db, Sigma0_VV_db
* periode

In [10]:
def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

In [13]:
def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data_values

# data_values

In [ ]:
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data

In [ ]:
bands_names = list(product_subset.getBandNames())[1:13]
cols = ['lat', 'lon', 'X', 'Y']
cols.extend(bands_names)
df_bands = pd.DataFrame(columns=cols)

In [ ]:
for i,r in tqdm(data.iterrows(), total data.shape[0]) :
    data = get_value(gc, data['lat'], data['long'])
    df_bands = df_bands.apapend(data)